In [4]:
# EDA 사전 세팅
%matplotlib inline

import time
import os
import openai
import folium

os.environ['KMP_WARNINGS'] = 'off'
from functools import reduce
import matplotlib as mpl
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

In [5]:
df = pd.read_csv("/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/의료기관_최종전처리.csv",  encoding='cp949')

In [6]:
dfn = pd.read_csv('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/주민등록인구통계.csv', encoding='cp949')

In [7]:
df

,일련번호,의료기관명,병상수,소 재 지,개설주체_민간 또는공공,개설일자,구,행정동명
0,3735,㈜LG창원1공장부속의원,0,경상남도 창원시 성산구 성산패총로 170 (가음정동),민간,1994-01-31,성산구,가음정동
1,322,국립마산병원,354,"경상남도 창원시 마산합포구 가포로 215 (가포동, 국립마산병원)",공공,1946-06-01,마산합포구,가포동
2,839,임재영피부과의원,0,"경상남도 창원시 진해구 충장로 361, 401호 (경화동, 그랜드메디컬)",민간,2009-12-24,진해구,경화동
3,845,명성마취통증의학과의원,0,경상남도 창원시 진해구 충장로 361 (경화동),민간,2007-12-10,진해구,경화동
4,851,청음이비인후과의원,0,"경상남도 창원시 진해구 충장로 361, 그랜드메디컬 2층 201호 (경화동)",민간,2006-04-11,진해구,경화동
...,...,...,...,...,...,...,...,...
1295,3118,명수한의원,0,"경상남도 창원시 마산회원구 회원남1길 49 (회원동, 삼성메르빌)",민간,2005-02-04,마산회원구,회원2동
1296,3125,장수한의원,0,"경상남도 창원시 마산회원구 회원동서로 34, 동화빌딩 (회원동)",민간,2002-07-25,마산회원구,회원2동
1297,3140,바로서한의원,0,"경상남도 창원시 마산회원구 북성로 106 (회원동, 환금프라자)",민간,1997-04-11,마산회원구,회원2동
1298,3141,임동진한의원,0,"경상남도 창원시 마산회원구 북성로 118 (회원동, 한효프라자 빌딩)",민간,1997-01-07,마산회원구,회원2동


In [8]:
dfh = pd.read_csv('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/행정동변경.csv')

In [9]:
dfh

,Unnamed: 0,법정동,행정동
0,0,명서동,명곡동
1,1,월남동,문화동
2,2,합성동,석전동
3,3,반지동,반송동
4,4,사림동,봉림동
5,5,팔용동,팔룡동
6,6,소계동,팔룡동
7,7,봉곡동,봉림동
8,8,남양동,남양동
9,9,서상동,의창동


In [10]:
dfn = dfn[['행정구역','노인총인구수']]

In [11]:
dfn['행정구역'] = dfn['행정구역'].apply(lambda x: x.split()[-1])

In [12]:
dfn

,행정구역,노인총인구수
0,동읍,6421
1,북면,9089
2,대산면,3342
3,의창동,11691
4,팔룡동,4831
5,명곡동,10661
6,봉림동,8696
7,반송동,8592
8,용지동,5947
9,중앙동,6441


In [24]:
dfn.to_csv('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/행정동별 노인인구 수.csv', encoding='cp949')

In [13]:
df1 = df['행정동명'].value_counts().reset_index()
df1.columns = ['행정동명', '의료기관 수']

print(df1)

     행정동명  의료기관 수
0     상남동     163
1     석전동     113
2     오동동      71
3     중앙동      66
4     팔룡동      63
5     내서읍      60
6     의창동      50
7     사파동      47
8     봉림동      43
9     명곡동      37
10   웅동2동      37
11    용지동      36
12   양덕2동      36
13     석동      32
14    반송동      30
15   회원2동      29
16    충무동      28
17    월용동      27
18     북면      23
19    산호동      22
20    남양동      20
21    풍호동      20
22   가음정동      19
23    경화동      17
24    진동면      17
25    문화동      17
26    여좌동      17
27    자은동      16
28     동읍      16
29    성주동      15
30    월영동      13
31     이동      13
32    웅남동      10
33    자산동      10
34   구암2동       9
35     현동       9
36    웅천동       7
37    회성동       6
38    완월동       6
39   웅동1동       5
40    교방동       5
41  반월중앙동       4
42    대산면       4
43    봉암동       3
44    태평동       2
45   용동2동       2
46    덕산동       2
47    구산면       1
48    장천동       1
49    가포동       1


In [14]:
merged_df = pd.merge(dfn, df1, left_on='행정구역', right_on='행정동명', how='left')

print(merged_df)

     행정구역  노인총인구수   행정동명  의료기관 수
0      동읍    6421     동읍    16.0
1      북면    9089     북면    23.0
2     대산면    3342    대산면     4.0
3     의창동   11691    의창동    50.0
4     팔룡동    4831    팔룡동    63.0
5     명곡동   10661    명곡동    37.0
6     봉림동    8696    봉림동    43.0
7     반송동    8592    반송동    30.0
8     용지동    5947    용지동    36.0
9     중앙동    6441    중앙동    66.0
10    상남동    5316    상남동   163.0
11    사파동    9431    사파동    47.0
12   가음정동    7488   가음정동    19.0
13    성주동    4547    성주동    15.0
14    웅남동    2192    웅남동    10.0
15    구산면    2359    구산면     1.0
16    진동면    4009    진동면    17.0
17    진북면    1880    NaN     NaN
18    진전면    2417    NaN     NaN
19     현동    3137     현동     9.0
20    가포동     654    가포동     1.0
21    월영동    9688    월영동    13.0
22    문화동    4071    문화동    17.0
23  반월중앙동    4967  반월중앙동     4.0
24    완월동    3387    완월동     6.0
25    자산동    3890    자산동    10.0
26    교방동    5240    교방동     5.0
27    오동동    7207    오동동    71.0
28    합포동    2632    NaN     NaN
29    산호동 

In [15]:
merged_df

,행정구역,노인총인구수,행정동명,의료기관 수
0,동읍,6421,동읍,16.0
1,북면,9089,북면,23.0
2,대산면,3342,대산면,4.0
3,의창동,11691,의창동,50.0
4,팔룡동,4831,팔룡동,63.0
5,명곡동,10661,명곡동,37.0
6,봉림동,8696,봉림동,43.0
7,반송동,8592,반송동,30.0
8,용지동,5947,용지동,36.0
9,중앙동,6441,중앙동,66.0


In [16]:
merged_df.drop('행정동명', axis=1, inplace=True)

# null 값을 0으로 대체
merged_df.fillna(0, inplace=True)

In [17]:
merged_df

,행정구역,노인총인구수,의료기관 수
0,동읍,6421,16.0
1,북면,9089,23.0
2,대산면,3342,4.0
3,의창동,11691,50.0
4,팔룡동,4831,63.0
5,명곡동,10661,37.0
6,봉림동,8696,43.0
7,반송동,8592,30.0
8,용지동,5947,36.0
9,중앙동,6441,66.0


In [18]:
# 노인 인구 수 대비 병원 수 변수 추가
merged_df['1 의료기관 당 노인인구'] = merged_df['노인총인구수'] / merged_df['의료기관 수']


In [31]:
df= pd.read_excel('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/종속변수.xlsx')

In [22]:
merged_df.to_excel('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/종속변수.xlsx')

In [32]:
df

,Unnamed: 0,행정구역,노인총인구수,의료기관 수,1 의료기관 당 노인인구
0,0,동읍,6421,16,401.312500
1,1,북면,9089,23,395.173913
2,2,대산면,3342,4,835.500000
3,3,의창동,11691,50,233.820000
4,4,팔룡동,4831,63,76.682540
5,5,명곡동,10661,37,288.135135
6,6,봉림동,8696,43,202.232558
7,7,반송동,8592,30,286.400000
8,8,용지동,5947,36,165.194444
9,9,중앙동,6441,66,97.590909


In [34]:
df['1 의료기관 당 노인인구'] = df['노인총인구수'] / df['의료기관 수']

In [33]:
df.drop('1 의료기관 당 노인인구', axis=1, inplace=True)

In [36]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [38]:
df.to_excel('/Users/choejeehyuk/DNA/경남 빅데이터활용 공모전/종속변수최종.xlsx')